# Redis

[redis](https://redis.io) (Remote Dictionary Server) es un almacenamiento in-memory utilizado como una base de datos de pares clave-valor, cache y broker de mensajes.

Se puede configurar para que sea durable, pero todos los datos se mantienen en memoria. Esto lo hace útil para la stream layer de una arquitectura lambda o, en lineas generales, como hot storage.

In [1]:
import redis
r = redis.Redis(
    host='redis',
    username="default",
    password="redis",
    port=6379,
    decode_responses=False,
    ssl=False
)

In [2]:
r.ping()

True

# Almacenamiento de pares clave-valor



Insertamos y obtenemos un par clave/valor.

In [3]:
r.set("k1", "v1")
r.get("k1")

b'v1'

In [4]:
r.exists("k1")

1

In [5]:
r.set("k1", "v100")
r.get("k1")

b'v100'

Podemos darle un TTL a las claves.

In [6]:
import time
from datetime import datetime

r.setex("ttl10", 10, "dead key")
print(f"set at {datetime.now()}")
time.sleep(5)
print(f"get at {datetime.now()}: {r.get('ttl10')}")
time.sleep(6)
print(f"get at {datetime.now()}: {r.get('ttl10')}")

set at 2024-06-11 05:13:39.693419
get at 2024-06-11 05:13:44.694125: b'dead key'
get at 2024-06-11 05:13:50.695178: None


In [7]:
r.setex("ttl10", 10, "dead key")
r.ttl("ttl10")

10

Podemos insertar todos los pares de clave/valor de un diccionario

In [8]:
r.mset({
    "k1":1,
    "k2":2,
    "k3":3
})
r.get("k1"), r.get("k2"), r.get("k3")

(b'1', b'2', b'3')

O, mas facil:

In [9]:
r.mget("k1", "k2", "k3")

[b'1', b'2', b'3']

## Ejercicios
Leyendo la [documentación](https://redis.readthedocs.io/en/stable/commands.html):

1. Inserte una clave `ej:1:key1` con valor `100`
2. Disminuzca en 10 el valor de la clave del punto 1
3. Agregue expiración de 30 segundos a la clave del punto 1
4. Obtenga la clave del punto 1 y actualice su TTL por 60 segundos en una sola operación
5. Insertar las claves `ejercicio:1:key2` y `ej:5:key1`. Listar todas las claves que comienzan con el prefijo `ej:`.
6. Cree una lista bajo la clave `ej:6:clientes` e inserte los valores 1, 2 y 3 en una operación cada uno. Elimine el elemento a izquierda e inserte a izquierda el valor 0.
7. Cree un hash bajo la clave `ej:7:keyhash` e inserte `{'a':1, 'b':2}`

# PubSub

In [10]:
p = r.pubsub(ignore_subscribe_messages=True)
p.subscribe('channel1')
p.get_message()

In [11]:
r.publish("channel1", 100)

1

In [12]:
p.get_message()

In [13]:
r.publish("channel2", 200)

0

In [14]:
p.get_message()

{'type': 'message', 'pattern': None, 'channel': b'channel1', 'data': b'100'}

In [15]:
p.psubscribe("channel*")

In [16]:
r.publish("channel5", 200)

1

In [17]:
p.get_message()

Y para correrlo en un event loop:

In [18]:
p = r.pubsub(ignore_subscribe_messages=True)

def message_handler(m: dict):
    if int(m["data"]) > 100:
        print(f"handler: {m}")

p.psubscribe(**{"channel*":message_handler})
thread = p.run_in_thread()

In [19]:
r.publish("channel1", 1000)

handler: {'type': 'pmessage', 'pattern': b'channel*', 'channel': b'channel1', 'data': b'1000'}


1

In [20]:
r.publish("channel1", 10)

1

In [21]:
r.publish("channelx", 500)

handler: {'type': 'pmessage', 'pattern': b'channel*', 'channel': b'channelx', 'data': b'500'}


1

Every message read from a PubSub instance will be a dictionary with the following keys.

type: One of the following: ‘subscribe’, ‘unsubscribe’, ‘psubscribe’, ‘punsubscribe’, ‘message’, ‘pmessage’

channel: The channel [un]subscribed to or the channel a message was published to

pattern: The pattern that matched a published message’s channel. Will be None in all cases except for ‘pmessage’ types.

data: The message data. With [un]subscribe messages, this value will be the number of channels and patterns the connection is currently subscribed to. With [p]message messages, this value will be the actual published message.

## Ejercicios
1. Queremos saber que tan rapido leemos los elementos publicados en redis. Cree un handler que filtre todos los valores impares y calcule el tiempo promedio entre que se publicó un mensaje con un valor par y lo leyó el handler. Implemente un ciclo while que inserte números aleatorios y cada 10_000 números insertados, lea de redis el promedio hasta ese momento.

In [22]:
import random
from itertools import count

p = r.pubsub(ignore_subscribe_messages=True)

def message_handler(m: dict):
    ...


p.psubscribe(**{"*":message_handler})

thread = p.run_in_thread()

for t in count():
    x = random.randint(1, 1000)
    r.publish("ej:pubsub", x)
    ...
    if t % 10_000 == 0:
        ...  # mostrar tiempo promedio hasta el momento

943


# Modulos extra
- RedisBloom:
- RedisGraph: 
- RedisJSON:
- RediSearch:
- RedisTimeSeries: 

# References
- https://redis.io/docs/latest/develop/connect/clients/python/
- https://redis.readthedocs.io/en/stable/redismodules.html
- https://redis.io/blog/beyond-the-cache-with-python/